In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data_path = "/home/mdafifal.mamun/notebooks/triagerX/comp_train.csv"

In [3]:
df = pd.read_csv(data_path)

In [4]:
df.head()

,Unnamed: 0,issue_number,issue_title,description,issue_url,issue_state,creator,labels,owner,component,text
0,2217,6455,AttachAPI backwards compatibility broken?,"@pdbain-ibm Hey Peter, I just reimplemented th...",https://github.com/eclipse-openj9/openj9/issue...,closed,DanHeidinga,"question, prio:high, comp:vm",NaN,comp:vm,Bug Title: AttachAPI backwards compatibility b...
1,2553,7332,JBoss cached LUDCL failure,"As reported in ""DefaultContextServiceServletTe...",https://github.com/eclipse-openj9/openj9/issue...,closed,DanHeidinga,"comp:vm, doc:externals",theresa-m,comp:vm,Bug Title: JBoss cached LUDCL failure\nBug Des...
2,1757,4976,Some ROMclasses in Java 11 are not stored in t...,Found some ROMclasses in Java 11 are not store...,https://github.com/eclipse-openj9/openj9/issue...,closed,a7ehuo,"bug, comp:vm, userRaised",babsingh,comp:vm,Bug Title: Some ROMclasses in Java 11 are not ...
3,3807,10454,JDK16 Valhalla ValueTypeTests FAILED: testGCFl...,Failure link\nFrom an internal build `Test_ope...,https://github.com/eclipse-openj9/openj9/issue...,closed,JasonFengJ9,"comp:gc, project:valhalla, test failure",NaN,comp:gc,Bug Title: JDK16 Valhalla ValueTypeTests FAILE...
4,2205,6438,Support and enable arraycopy transformations f...,This issue tracks the work to enable the array...,https://github.com/eclipse-openj9/openj9/issue...,closed,0xdaryl,"help wanted, comp:jit, arch:aarch64",oussamasaoudi,comp:jit,Bug Title: Support and enable arraycopy transf...


In [10]:
df = df[df["component"].notna()]

In [11]:
# filtered_df = df[df["component"].isin(["comp:jvmti", "comp:jclextensions"])]

In [12]:
# filtered_df["component"].value_counts()

In [13]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer


class Llama3:
    def __init__(self, model_id, max_new_tokens=512):
        self.tokenizer = AutoTokenizer.from_pretrained(model_id)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_id,
            torch_dtype=torch.bfloat16,
            device_map="auto",
        )
        self.max_new_tokens = max_new_tokens

    def invoke(
        self, system_prompt, question, temperature=0.5, return_token_usage=False
    ):
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question},
        ]

        input_ids = self.tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            max_length=self.model.config.max_position_embeddings - self.max_new_tokens,
            truncation=True,
            return_tensors="pt",
        ).to(self.model.device)

        terminators = [
            self.tokenizer.eos_token_id,
            self.tokenizer.convert_tokens_to_ids("<|eot_id|>"),
        ]

        outputs = self.model.generate(
            input_ids,
            max_new_tokens=self.max_new_tokens,
            eos_token_id=terminators,
            do_sample=True,
            temperature=temperature,
        )
        response = outputs[0][input_ids.shape[-1] :]

        if return_token_usage:
            prompt_tokens = len(input_ids[0])
            completion_tokens = len(outputs[0]) - prompt_tokens

            token_usage = {
                "prompt_tokens": prompt_tokens,
                "completion_tokens": completion_tokens,
                "total_tokens": prompt_tokens + completion_tokens,
            }

            return (
                self.tokenizer.decode(response, skip_special_tokens=True),
                token_usage,
            )

        return self.tokenizer.decode(response, skip_special_tokens=True)


In [14]:
MODEL_ID = "meta-llama/Meta-Llama-3-8B-Instruct"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {device}")
llama3_model = Llama3(MODEL_ID)

Using device: cuda


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [28]:
SYSTEM_PROMPT = """You are an efficient and correct data augmentor. 
Your job is when provided a bug report text, you modify only the natural language part. 
You can use synonyms or change the structure of the sentence by swapping words. 
But you never change code artifacts like source codes, stack traces, logs, crash reports, version, etc.
Only return the augmented output. Nothing else.

Now augment the following bug report:
"""

In [51]:
filtered_df.iloc[13]

Unnamed: 0                                                   5729
issue_number                                                14662
issue_title                               Security Model for CRIU
description     CRIU checkpoint/restore jdk feature requires s...
issue_url       https://github.com/eclipse-openj9/openj9/issue...
issue_state                                                  open
creator                                                   alon-sh
labels                             comp:jclextensions, criu, beta
owner                                                     alon-sh
component                                      comp:jclextensions
text            Bug Title: Security Model for CRIU\nBug Descri...
Name: 1171, dtype: object

In [52]:
idx = 19

bug_report = filtered_df.iloc[idx].text
print("Component:", filtered_df.iloc[idx].component)
print(filtered_df.iloc[idx].text)
print("##############################")
print(llama3_model.invoke(system_prompt=SYSTEM_PROMPT, question=f"Bug Report: {bug_report}"))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Component: comp:jvmti
Bug Title: RedefinePreviousVersions not supported by OpenJ9
Bug Description: `serviceability/jvmti/RedefineClasses/RedefinePreviousVersions.java` tests Hotspot specific cmd line options which
are not supported in OpenJ9:
-Xlog:redefine+class+iklass+add=trace,redefine+class+iklass+purge=trace
Test source code: Setting `vm.opt.final.ClassUnloading` to true enables this test.
This test will be permanently disabled for OpenJ9 since the above options are not supported by OpenJ9.
##############################
Bug Report: Bug Title: RedefinePreviousVersions not compatible with OpenJ9
Bug Description: `serviceability/jvmti/RedefineClasses/RedefinePreviousVersions.java` validates Hotspot-specific command-line options that are incompatible with OpenJ9:
-Xlog:redefine+class+iklass+add=trace,redefine+class+iklass+purge=trace
Test source code: Enabling `vm.opt.final.ClassUnloading` to true enables this test.
This test will be permanently disabled for OpenJ9 as these options a

In [15]:
from tqdm import tqdm

In [16]:
print(len(df))

5004


In [65]:
len(augmented_rows)

67

In [29]:
augmented_rows = []

for component, num_components in df["component"].value_counts().items():
    if num_components < 1000:
        print(f"Augmenting data for component: {component}")
        augmentation_needed = 1000 - num_components

        # Filter rows for the current component
        original_data = df[df["component"] == component]

        # Augment data by sampling with replacement
        for _ in tqdm(range(augmentation_needed), total=augmentation_needed):
            # Sample one row randomly from the original_data
            row = original_data.sample(n=1, replace=True).iloc[0]
            print("Original data:", row["text"])
            bug_report = row["text"]
            augmented_text = llama3_model.invoke(system_prompt=SYSTEM_PROMPT, question=f"Bug Report: {bug_report}")
            augmented_row = row.copy()
            augmented_row["text"] = augmented_text
            print("Augmented data:", augmented_text)

            augmented_rows.append(augmented_row)

Augmenting data for component: comp:test


  0%|          | 0/438 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Original data: Bug Title: Produce a summarized excludes list for a particular test run
Bug Description: We currently run several different types of tests (that use various underlying test frameworks). There are several methods for excluding test targets and the individual tests within the test target. We would like a way to summarize all of the test exclusions of a particular test run. There are a variety of ways to do this, one way, which would produce this info even on a manual/local test run would be to add a "getExcludes" target to testkitgen to tally test exclusions.
Different supported mechanisms to exclude tests:
- disable an entire test target in playlist.xml using `<disabled>` tag (preferred way)
- disable a target's subtest: openjdk regression test using openjdk exclude files - disable a target's subtest: testNG functional test using functional exclude files
- disable a target's subtest: cmdlinetester functional test


  0%|          | 1/438 [00:05<36:48,  5.05s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Augmented data: Bug Report: Bug Title: Generate a concise summary of excluded tests for a specific test execution

Bug Description: We execute multiple types of tests (utilizing various underlying test frameworks). There are several approaches to excluding test targets and individual tests within the target. We require a method to summarize all test exclusions for a particular test run. One feasible approach would be to incorporate a "getExcludes" target into testkitgen to track test exclusions, even during manual or local test execution.

Supported mechanisms to exclude tests:

- Preferably, disable an entire test target in playlist.xml using the `<disabled>` tag
- Disable a target's subtest: openjdk regression test using openjdk exclusion files
- Disable a target's subtest: testNG functional test using functional exclusion files
- Disable a target's subtest: cmdlinetester functional test
Original data: Bug Title: Investigate use of public AzDo in combo with Azure VM agents Jenkins pl

...

Augmented data: Bug Report: Bug Title: Class.getMethods assessment is incomplete
Bug Description: Class.getMethods are not triggered by verification tests. org.openj9.test.reflect.GetMethodsAssessment has been developed. Appears to be just a missing line `<class name="org.openj9.test.reflect.GetMethodsAssessment" />` in file test/functional/Java8andUp/testng.xml
Original data: Bug Title: OpenSSL testing to confirm OpenSSL natives are being loaded
Bug Description: As per OpenSSL support may be silently failing. There should be a test, likely using the `-Djdk.nativeCryptoTrace=true` option, to confirm the natives are being loaded as expected.


  3%|▎         | 15/438 [01:23<24:30,  3.48s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Augmented data: Bug Report: Bug Title: Verifying OpenSSL Natives for Native Crypto Loading
Bug Description: According to OpenSSL documentation, silent failures may occur. To ensure native loading, a test is required, possibly utilizing the `-Djdk.nativeCryptoTrace=true` flag, to verify the natives are being loaded as intended.
Original data: Bug Title: ValueTypes: java.lang.VerifyError in ValueTypeTests_1  testFlatLayoutsWithRecursiveLongs
Bug Description: After removing `<class name="org.openj9.test.lworld.ValueTypeArrayTests"/>` from `testng.xml`, I'm able to run more Valhalla functional tests. The reason that it has to be removed in order to run more tests is tracked in `testFlatLayoutsWithRecursiveLongs` in `ValueTypeTests_1` failed due to `"java.lang.VerifyError: JVMVRFY021 thrown object not throwable; class=QuadLong, method=withl(QDoubleLong;)QQuadLong;, pc=2"`
1:28:13 Eclipse OpenJ9 VM (build test-fix-field-offset-flattened-arrayelement-bb482f0ec7b, JRE 20 Linux amd64-64-Bit Com

  3%|▎         | 15/438 [01:28<41:34,  5.90s/it]


KeyboardInterrupt: 

In [69]:
for i, row in tqdm(filtered_df.iterrows(), total=len(filtered_df)):
    bug_report = row["text"]
    augmented_text = llama3_model.invoke(system_prompt=SYSTEM_PROMPT, question=f"Bug Report: {bug_report}")
    augmented_row = row.copy()
    augmented_row["text"] = augmented_text

    augmented_rows.append(augmented_row)

  0%|          | 0/67 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
  1%|▏         | 1/67 [00:37<41:12, 37.46s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
  3%|▎         | 2/67 [00:43<20:17, 18.73s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
  4%|▍         | 3/67 [01:17<27:37, 25.90s/it]The attention mask and the pad token id were not set. As a consequen

In [58]:
augmented_rows[-1]

Unnamed: 0                                                   6815
issue_number                                                16970
issue_title     JDK20  jdk_nio_0_FAILED UnsupportedOperationEx...
description     Failure link\nFrom [an internal build](\n open...
issue_url       https://github.com/eclipse-openj9/openj9/issue...
issue_state                                                closed
creator                                               JasonFengJ9
labels            test failure, os:aix, comp:jclextensions, jdk20
owner                                                backwaterred
component                                      comp:jclextensions
text            Bug Report: Bug Title: JDK20  jdk_nio_0_FAILED...
Name: 4968, dtype: object

In [70]:
augmented_df = pd.DataFrame(augmented_rows)
result_df = pd.concat([df, augmented_df], ignore_index=True)

In [71]:
len(result_df)

5205

'Bug Title: Verifying OpenSSL Natives for Native Crypto Loading\nBug Description: According to OpenSSL documentation, silent failures may occur. To ensure native loading, a test is required, possibly utilizing the `-Djdk.nativeCryptoTrace=true` flag, to verify the natives are being loaded as intended.'

In [72]:
result_df.component.value_counts()

component
comp:vm               1698
comp:jit              1354
comp:test              562
comp:build             378
comp:jitserver         373
comp:infra             329
comp:gc                243
comp:jclextensions     180
comp:jvmti              88
Name: count, dtype: int64

In [73]:
result_df.to_csv("comp_train_augmented_3x.csv")

In [64]:
result_df.iloc[-5]

Unnamed: 0                                                   8217
issue_number                                                20324
issue_title     On AIX using zlibNX may cause ClassNotFoundExc...
description     Experienced the following problem on AIX 7.3 P...
issue_url       https://github.com/eclipse-openj9/openj9/issue...
issue_state                                                  open
creator                                                  pshipton
labels                                 os:aix, comp:jclextensions
owner                                                     r30shah
component                                      comp:jclextensions
text            Bug Report: Bug Title: On AIX utilizing zlibNX...
Name: 5066, dtype: object